## Setup and Imports

In this section, we import all the necessary Python libraries used throughout the analysis:

- **scikit-learn** for preprocessing, clustering, classification, and evaluation  
- **pandas/numpy** for data manipulation  
- **matplotlib/seaborn/plotly** for visualization  
- **joblib** for saving machine-learning models  

We also configure matplotlib for consistent plotting inside Google Colab.


In [1]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

In [ ]:
# Colab-specific installs (run if packages missing)
!pip install --quiet kaggle scikit-learn pandas matplotlib seaborn plotly


# Standard imports
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans, DBSCAN
from sklearn.metrics import silhouette_score, davies_bouldin_score, confusion_matrix, classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
import joblib


# Plot settings
%matplotlib inline
plt.rcParams['figure.figsize'] = (8,5)

##  Loading the Weather Dataset

We load the raw weather dataset from disk to begin our analysis.
The dataset contains several meteorological measurements and a binary label indicating whether it rains or not.

We display the full dataframe to visually inspect the structure.


In [ ]:
df = pd.read_csv('/content/weather_forecast_data.csv')

In [ ]:
df

##  Dataset Structure and Missing Values

Here we examine:

- column names and data types  
- number of rows  
- memory usage  
- how many missing values each column contains  

This helps determine preprocessing steps such as imputation or cleaning.


In [ ]:
# Summary of columns and types
df.info()

# Count missing values
df.isnull().sum()

##  Outlier Visualization (Before Cleaning)

A boxplot is used to visualize the distribution of each numeric feature and detect potential outliers.

Outliers may negatively affect scaling, PCA, and clustering performance, so identifying them early is important.


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(12, 6))
df.boxplot(rot=45)
plt.title("Boxplot of Weather Dataset Features (Before Outlier Removal)")
plt.show()


##  Removing Outliers Using Z-Score

We compute Z-scores for all numeric columns.
Values with |Z| > 3 are considered statistical outliers.

We then filter out any rows containing such extreme values.

This improves overall clustering quality and stabilizes model training.


In [ ]:
from scipy import stats
import numpy as np

# Select only numeric columns
num_df = df.select_dtypes(include=['float64', 'int64'])

# Compute Z-scores
z_scores = np.abs(stats.zscore(num_df))

# Rows where all z-scores < 3
df_no_outliers = df[(z_scores < 3).all(axis=1)]

df_no_outliers.shape


##  Dataset Size Before and After Cleaning

We compare the original dataset size with the cleaned one
to understand how many outlier rows were removed.


In [ ]:
print("Original dataset shape:", df.shape)
print("After removing outliers:", df_no_outliers.shape)


##  Outlier Visualization (After Cleaning)

We re-plot the boxplots to ensure that extreme values have been effectively removed.

This confirms that our dataset is now suitable for scaling and clustering.


In [ ]:
plt.figure(figsize=(12, 6))
df_no_outliers.boxplot(rot=45)
plt.title("Boxplot After Outlier Removal")
plt.show()


##  Extracting the Target Variable

The `Rain` column is the binary classification target.

We store it as **y** and remove it from the feature set.
Only the remaining meteorological features are used for clustering and modeling.


In [ ]:
y = df['Rain']

# Drop the target column from the dataset
df = df.drop(columns=['Rain'])
y

##  Feature Scaling (Standardization)

We apply **StandardScaler** to normalize the numeric features.

Scaling ensures:
- all features contribute equally  
- PCA and K-Means behave correctly  
- distance-based models do not become biased by large-scale features


In [ ]:
from sklearn.preprocessing import StandardScaler

# Select numeric columns
num_cols = df_no_outliers.select_dtypes(include=['float64', 'int64']).columns

# Initialize the scaler
scaler = StandardScaler()

# Fit and transform
scaled_data = scaler.fit_transform(df_no_outliers[num_cols])

# Convert back to DataFrame for easier use
df_scaled = pd.DataFrame(scaled_data, columns=num_cols)

df_scaled.head()


##  PCA Dimensionality Reduction (Unclustered Data)

PCA is used to project high-dimensional weather data into 2D.

This does NOT perform clustering — it simply helps visualize
the structure of the dataset before applying algorithms.


In [ ]:
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

pca = PCA(n_components=2)
pca_result = pca.fit_transform(df_scaled)

plt.figure(figsize=(8,6))
plt.scatter(pca_result[:, 0], pca_result[:, 1], s=5)
plt.title("PCA Projection of Weather Data (Unclustered)")
plt.xlabel("PC1")
plt.ylabel("PC2")
plt.show()


##  Feature Engineering

We create new derived features to capture additional relationships:

- **Wind_Power** (energy approximation from wind speed)
- **Humidity_Cloud** (interaction term)
- **Pressure_Change** (temporal weather trend)
- **Humidity_to_Pressure** and **Cloud_to_Pressure** (ratios)

Engineered features often improve clustering and classification performance.


In [ ]:
import pandas as pd

# Rename messy column names for easier coding
df = df.rename(columns={
    'Temperatu': 'Temperature',
    'Wind_Spee': 'Wind_Speed',
    'Cloud_Cov': 'Cloud_Cover'
})

# ----- 1. Wind Power -----
df['Wind_Power'] = 0.5 * (df['Wind_Speed'] ** 2)

# ----- 2. Humidity × Cloud Interaction -----
df['Humidity_Cloud'] = df['Humidity'] * df['Cloud_Cover']

# ----- 3. Pressure Drop (difference from previous day) -----
df['Pressure_Change'] = df['Pressure'].diff().fillna(0)

# ----- 4. Ratios -----
df['Humidity_to_Pressure'] = df['Humidity'] / (df['Pressure'] + 1e-5)
df['Cloud_to_Pressure'] = df['Cloud_Cover'] / (df['Pressure'] + 1e-5)

df.head()

##  Rescaling After Feature Creation

After generating new features, we re-apply StandardScaler
to ensure the entire feature set remains normalized.

This step is necessary because new columns shift the feature distribution.


In [ ]:
from sklearn.preprocessing import StandardScaler

numeric_cols = df.select_dtypes(include=['float64', 'int64']).columns

scaler = StandardScaler()
df_scaled = pd.DataFrame(scaler.fit_transform(df[numeric_cols]), columns=numeric_cols)

df_scaled.head()

#KMEANS

##  **K-Means Elbow Method**

We test values of **k = 1 to 9** and compute inertia for each model.

The elbow point shows where adding more clusters stops improving the model significantly.

This helps select the optimal number of clusters.


In [ ]:
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

inertia = []
K_range = range(1, 10)

for k in K_range:
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(df)
    inertia.append(kmeans.inertia_)

plt.plot(K_range, inertia, marker='o')
plt.xlabel('Number of clusters (k)')
plt.ylabel('Inertia')
plt.title('Elbow Method')
plt.show()


##  Silhouette Score for Cluster Quality

We compute silhouette scores for k = 2 to 9.

The silhouette score measures:
- how compact clusters are  
- how separated they are  

This helps us confirm the ideal number of clusters.


In [ ]:
from sklearn.metrics import silhouette_score

sil_scores = {}

for k in range(2, 10):
    kmeans = KMeans(n_clusters=k, random_state=42)
    labels = kmeans.fit_predict(df)
    sil_scores[k] = silhouette_score(df, labels)

sil_scores

##  Applying K-Means Clustering

We select **k = 2** based on evaluation metrics.

Cluster labels are added back to the dataframe, allowing us to visualize
and analyze the characteristics of each cluster.


In [ ]:
best_k = 2
kmeans = KMeans(n_clusters=best_k, random_state=42)
df['Cluster'] = kmeans.fit_predict(df)

df[['Temperature','Humidity','Cloud_Cover','Cluster']].head()


##  K-Means Visualization (PCA)

We plot the K-Means clusters using PCA-reduced data.

This 2D visualization helps us understand the separation between clusters
and identify grouping patterns in the weather data.


In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
pca_data = pca.fit_transform(df)

plt.scatter(pca_data[:,0], pca_data[:,1], c=df['Cluster'], s=8)
plt.title("K-Means Clusters (PCA Projection)")
plt.xlabel("PC1")
plt.ylabel("PC2")
plt.show()


##  Pairplot Visualization of Clustered Features

A pairplot reveals relationships between features for each cluster.

This helps uncover:
- feature correlations  
- cluster shapes  
- overlapping regions  


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.pairplot(df, hue='Cluster', diag_kind='kde', markers='o', height=2)
plt.suptitle("Pairplot of Features by Cluster", y=1.02)
plt.show()


##  Boxplots of Features per Cluster

Boxplots show how each cluster differs statistically across key features.

This is important for interpreting weather "types" created by K-Means.


In [ ]:
import matplotlib.pyplot as plt

features = ['Temperature','Humidity','Wind_Speed','Cloud_Cover','Pressure']

plt.figure(figsize=(14, 10))

for i, col in enumerate(features, 1):
    plt.subplot(3, 2, i)
    sns.boxplot(data=df, x='Cluster', y=col)
    plt.title(f"{col} by Cluster")

plt.tight_layout()
plt.show()


##  Cluster Centroid Heatmap

We compute the mean of each feature per cluster and plot it in a heatmap.

This acts as a "fingerprint" for each cluster and shows what weather pattern it represents.


In [ ]:
import pandas as pd

# Calculate centroids from dataset
centroids = df.groupby("Cluster").mean()

plt.figure(figsize=(10, 6))
sns.heatmap(centroids, annot=True, cmap="viridis")
plt.title("Cluster Centroids (Mean Values)")
plt.show()


##  3D PCA Visualization

A 3-component PCA is used to visualize the clustered data in 3D space.

This offers additional insight into how the datapoints separate beyond 2D.


In [ ]:
from sklearn.decomposition import PCA
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt

pca_3d = PCA(n_components=3)
pca_data = pca_3d.fit_transform(df)

fig = plt.figure(figsize=(8, 6))
ax = fig.add_subplot(111, projection='3d')

ax.scatter(pca_data[:,0], pca_data[:,1], pca_data[:,2],
           c=df['Cluster'], s=20)

ax.set_xlabel("PC1")
ax.set_ylabel("PC2")
ax.set_zlabel("PC3")
plt.title("3D PCA K-Means Visualization")
plt.show()


##  Radar Charts of Cluster Profiles

Radar charts display each feature's normalized contribution within each cluster.

They provide an intuitive shape-based comparison of weather types.


In [ ]:
import numpy as np

centroids = df.groupby("Cluster").mean()

# Normalize values for nicer plotting
centroids_norm = centroids / centroids.max()

labels = centroids_norm.columns
num_vars = len(labels)

for cluster_id in centroids_norm.index:
    values = centroids_norm.loc[cluster_id].values.tolist()
    values += values[:1]

    angles = np.linspace(0, 2*np.pi, num_vars, endpoint=False).tolist()
    angles += angles[:1]

    plt.figure(figsize=(6,6))
    plt.polar(angles, values, marker='o')
    plt.fill(angles, values, alpha=0.25)
    print("")
    plt.title(f"Radar Chart for Cluster {cluster_id}")

    plt.xticks(angles[:-1], labels)
print("")
plt.show()



#DBSCAN

## **DBSCAN Clustering (Why It Is Not Useful Here)**

We attempt DBSCAN, a density-based clustering algorithm.
However, DBSCAN is **not suitable for this dataset** for several reasons:

###  1. DBSCAN expects natural dense clusters
Weather data is continuous, smooth, and evenly distributed →  
no dense regions separated by sparse gaps.

###  2. DBSCAN detects noise, not classification patterns
Our target variable is **binary classification (Rain / No Rain)**.  
DBSCAN is unsupervised → it cannot learn decision boundaries.

###  3. DBSCAN assigns most points to one giant cluster
Because the dataset has no natural density separations,
DBSCAN frequently results in:

- one large cluster  
- all other points labeled as noise (-1)

###  4. Silhouette and Davies–Bouldin fail
When DBSCAN produces 0 or 1 clusters, evaluation metrics cannot be computed.

###  Conclusion  
**DBSCAN is not appropriate** for weather forecasting datasets with smooth,
continuous numeric distributions.  
Therefore, we do not continue clustering with DBSCAN.


In [ ]:
from sklearn.cluster import DBSCAN

dbscan = DBSCAN(eps=0.5, min_samples=5)
db_labels = dbscan.fit_predict(df)

df['DBSCAN_Cluster'] = db_labels
df[['DBSCAN_Cluster']].head()


In [ ]:
import numpy as np

unique, counts = np.unique(db_labels, return_counts=True)
dict(zip(unique, counts))


##  PCA Visualization of DBSCAN Results

We still plot the DBSCAN result to confirm the failure mode:
- all points assigned to one huge cluster  
- anomalies labeled as -1  


In [ ]:
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

pca = PCA(n_components=2)
pca_data = pca.fit_transform(df_scaled)

plt.scatter(pca_data[:,0], pca_data[:,1], c=db_labels, cmap='plasma', s=10)
plt.title("DBSCAN Clusters (PCA 2D Projection)")
plt.xlabel("PC1")
plt.ylabel("PC2")
plt.show()

##  DBSCAN Evaluation Attempt

DBSCAN often produces too few clusters to evaluate.

If the number of detected clusters < 2,
Silhouette Score and Davies–Bouldin cannot be computed.

This confirms DBSCAN is not effective for this dataset.


In [ ]:
from sklearn.metrics import silhouette_score, davies_bouldin_score

# Filter out anomalies (-1)
mask = db_labels != -1
if len(set(db_labels[mask])) > 1:
    sil = silhouette_score(X[mask], db_labels[mask])
    dbi = davies_bouldin_score(X[mask], db_labels[mask])
    print("Silhouette Score:", sil)
    print("Davies-Bouldin Index:", dbi)
else:
    print("Not enough clusters for silhouette/DBI evaluation.")


##  **Future Work**

Based on the current project progression and methodology, several improvements can be made to deepen the analysis and increase the predictive capability of the system:

###  1. Improved Feature Engineering
- Add *time-based features* such as:
  - Lagged temperature/humidity values (t−1, t−2)
  - Rolling mean / rolling variance
  - Seasonal indicators (month, day-of-year, weekend flag)
- Derive *meteorological indices*:
  - Dew point
  - Heat index
  - Real-feel temperature
  - Wind chill factor
- Build weather interaction features:
  - Temperature × Humidity  
  - Pressure × Wind speed  

These features could significantly improve classification performance.

---

###  2. Advanced Unsupervised Learning
- Apply **HDBSCAN**, which handles variable-density clusters better than DBSCAN.
- Use **Gaussian Mixture Models (GMM)** to learn soft clusters (probabilistic weather types).
- Explore **hierarchical clustering** to see multi-level weather groupings.

This may reveal more meaningful weather “types” beyond the hard partitions from K-Means.

---

###  3. Stronger Supervised Models
Using the K-Means clusters or real “Rain/No Rain” labels:

- Evaluate **Gradient Boosting models** (XGBoost, CatBoost, LightGBM)
- Evaluate **SVM with RBF kernel** for nonlinear decision boundaries  
- Apply **Neural Models**:
  - MLP with more hidden layers
  - 1D CNN for time-based / sensor weather sequences  
  - LSTM/GRU for sequential weather predictions  

These models typically outperform classical methods on structured weather data.

---

###  4. Time-Series Modeling  
Since weather is inherently sequential:

- Apply **ARIMA/SARIMA** to forecast temperature/pressure trends  
- Use **Prophet** for seasonal decomposition  
- Train **LSTM/Transformer models** to predict future weather conditions  

This shifts the project from static classification to real forecasting.

---

###  5. Better Evaluation and Interpretability
- SHAP values for model interpretability  
- Partial Dependence Plots (PDPs)  
- Weather cluster semantic labeling (e.g., "Hot–Dry", "Humid–Stormy")  

These provide more descriptive cluster meanings rather than just numbers.

---

###  6. Real-World Deployment
- Build a small **Streamlit dashboard** to visualize:
  - Clusters  
  - Forecasts  
  - Daily predictions  
- Export predictions as a simple REST API  
- Automate daily data collection and prediction via cron jobs  

This makes the project practical and interactive.

---

##  Summary  
These extensions would make the project more comprehensive, more predictive, and more aligned with real-world weather analysis tasks.


In [ ]:
df

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

# choose numeric features for clustering (exclude label)
numeric_cols = df.select_dtypes(include=['float64','int64']).columns.tolist()
numeric_cols = [c for c in numeric_cols if c != 'Rain_Encoded']

X = df[numeric_cols].copy()

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

pca = PCA(n_components=2, random_state=42)
X_pca = pca.fit_transform(X_scaled)

# quick sanity
print("Scaled shape:", X_scaled.shape)

In [ ]:
df

In [ ]:
from sklearn.mixture import GaussianMixture
from sklearn.metrics import silhouette_score, davies_bouldin_score
import numpy as np
import matplotlib.pyplot as plt

# try GMM for k=1..6 (adjust upper bound if you want)
k_range = range(1, 7)
bics, aics = [], []
gmm_models = {}

for k in k_range:
    print("Fitting GMM with k =", k)
    gmm = GaussianMixture(n_components=k, covariance_type='full', random_state=42)
    gmm.fit(X_scaled)
    bics.append(gmm.bic(X_scaled))
    aics.append(gmm.aic(X_scaled))
    gmm_models[k] = gmm

# Plot BIC and AIC (each is its own plot)
plt.figure(figsize=(8,4))
plt.plot(list(k_range), bics, marker='o')
plt.title('GMM BIC vs #components')
plt.xlabel('n_components')
plt.ylabel('BIC')
plt.show()

plt.figure(figsize=(8,4))
plt.plot(list(k_range), aics, marker='o')
plt.title('GMM AIC vs #components')
plt.xlabel('n_components')
plt.ylabel('AIC')
plt.show()

# Pick best k by lowest BIC (common choice)
best_k = int(k_range[np.argmin(bics)])
print("Chosen best_k (lowest BIC):", best_k)

best_gmm = gmm_models[best_k]
gmm_labels = best_gmm.predict(X_scaled)        # hard assignments for metrics
gmm_probs = best_gmm.predict_proba(X_scaled)   # soft responsibilities

# Evaluate (silhouette + Davies-Bouldin) using hard labels if k>=2
if best_k >= 2:
    sil = silhouette_score(X_scaled, gmm_labels)
    dbi = davies_bouldin_score(X_scaled, gmm_labels)
else:
    sil, dbi = np.nan, np.nan

print(f"GMM metrics (k={best_k}): silhouette={sil:.4f}, davies_bouldin={dbi:.4f}")

In [ ]:
# PCA scatter (each point colored by its GMM hard label)
plt.figure(figsize=(8,6))
plt.scatter(X_pca[:,0], X_pca[:,1], c=gmm_labels, s=20)
plt.title(f'GMM hard labels on PCA (k={best_k})')
plt.xlabel('PC1')
plt.ylabel('PC2')
plt.show()

In [ ]:
# stacked bar plot of responsibilities for the first up to 200 records
n_display = min(200, gmm_probs.shape[0])
indices = np.arange(n_display)
bottom = np.zeros(n_display)

plt.figure(figsize=(12,4))
for comp in range(best_k):
    plt.bar(indices, gmm_probs[:n_display, comp], bottom=bottom)
    bottom += gmm_probs[:n_display, comp]
plt.title(f'GMM soft assignments (first {n_display} samples) — stacked responsibilities')
plt.xlabel('sample index (first rows)')
plt.ylabel('probability mass')
plt.show()

In [ ]:
# component means are in scaled space — inverse transform to original scale
centers_scaled = best_gmm.means_
centers_orig = scaler.inverse_transform(centers_scaled)

plt.figure(figsize=(10, max(2, best_k*0.5)))
plt.imshow(centers_orig, aspect='auto')
plt.yticks(range(best_k), [f'Comp {i}' for i in range(best_k)])
plt.xticks(range(len(numeric_cols)), numeric_cols, rotation=90)
plt.title('GMM component means (original feature scale)')
plt.colorbar()
plt.show()

In [ ]:
from scipy.cluster import hierarchy
from sklearn.cluster import AgglomerativeClustering
import matplotlib.pyplot as plt

# compute linkage (Ward)
linked = hierarchy.linkage(X_scaled, method='ward')

# Dendrogram: truncated to last 150 merges for readability if dataset is large
plt.figure(figsize=(12,6))
hierarchy.dendrogram(linked, truncate_mode='lastp', p=150)
plt.title('Hierarchical Clustering Dendrogram (truncated to last 150 merges)')
plt.xlabel('Cluster size')
plt.ylabel('Distance')
plt.show()

# Choose number of clusters for Agglomerative (use best_k as starting point)
n_agg = max(2, best_k)
agg = AgglomerativeClustering(n_clusters=n_agg)
agg_labels = agg.fit_predict(X_scaled)

plt.figure(figsize=(8,6))
plt.scatter(X_pca[:,0], X_pca[:,1], c=agg_labels, s=20)
plt.title(f'Agglomerative Clustering (n_clusters={n_agg}) — PCA 2D')
plt.xlabel('PC1')
plt.ylabel('PC2')
plt.show()

In [ ]:
df['GMM_Label'] = gmm_labels
df['Agg_Label'] = agg_labels
df.to_csv('/content//weather_with_advanced_clusters.csv', index=False)
print("Saved: /content/weather_with_advanced_clusters.csv")

In [ ]:
df = pd.read_csv('/content/weather_with_advanced_clusters.csv')

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

In [ ]:
y

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression(max_iter=1000)
logreg.fit(X_train_scaled, y_train)

In [ ]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

y_pred = logreg.predict(X_test_scaled)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

cm = confusion_matrix(y_test, y_pred)

plt.figure(figsize=(5,5))
plt.imshow(cm, interpolation='nearest')
plt.title("Confusion Matrix - Logistic Regression")
plt.colorbar()
ticks = np.arange(len(cm))
plt.xticks([0,1], ['No Rain','Rain'])
plt.yticks([0,1], ['No Rain','Rain'])
plt.xlabel('Predicted')
plt.ylabel('Actual')

for i in range(cm.shape[0]):
    for j in range(cm.shape[1]):
        plt.text(j, i, cm[i, j], ha="center", va="center")

plt.show()

In [ ]:
# 1) Prepare the Rain target and attach to df if necessary, then train Decision Tree
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
import joblib

# load dataframe (use enriched file if present)
path_enriched = '/content/weather_with_advanced_clusters.csv'
path_orig = '/content/weather_forecast_data.csv'
if os.path.exists(path_enriched):
    df = pd.read_csv(path_enriched)
    print("Loaded:", path_enriched)
elif os.path.exists(path_orig):
    df = pd.read_csv(path_orig)
    print("Loaded:", path_orig)
else:
    raise FileNotFoundError("No dataset found under /content. Upload your CSV there.")

try:
    # 'y' may exist in the notebook as a variable — attach if so and if df lacks 'Rain'
    if 'y' in globals() and 'Rain' not in df.columns:
        # ensure index alignment: if y is same length and integer index, assign directly
        if len(y) == len(df):
            df['Rain'] = y.values
            print("Attached existing 'y' Series to df as df['Rain'].")
        else:
            print("Found 'y' Series but length != df. Skipping automatic attach. You may need to align indices.")
except Exception as e:
    print("Skipping attach of 'y' due to:", e)

# If df has a Rain column (strings like 'rain'/'no rain'), encode it to numeric
if 'Rain' in df.columns:
    df['Rain_Encoded'] = df['Rain'].map({'rain':1, 'no rain':0})
    if df['Rain_Encoded'].isnull().any():
        # if mapping didn't cover all values, try more robust encoding
        df['Rain_Encoded'] = df['Rain'].astype('category').cat.codes
    print("Created 'Rain_Encoded' from df['Rain']. Unique values:", df['Rain_Encoded'].unique())
else:
    raise ValueError("No 'Rain' column found in dataframe and no 'y' attached. Please attach your rain labels as df['Rain'].")

# --- Prepare features and target ---
target = 'Rain_Encoded'
X = df.drop(columns=[target, 'Rain'], errors='ignore')
X = X.select_dtypes(include=[np.number])   # keep numeric features only
y_enc = df[target].astype(int)

print("Using features:", X.columns.tolist())
print("Target distribution:\n", y_enc.value_counts(normalize=False))

# --- Train/test split ---
X_train, X_test, y_train, y_test = train_test_split(
    X, y_enc, test_size=0.2, random_state=42, stratify=y_enc
)

# --- Scaling (optional for tree but consistent across models) ---
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# --- Train Decision Tree ---
dt = DecisionTreeClassifier(random_state=42)
dt.fit(X_train_scaled, y_train)

# --- Evaluate ---
y_pred = dt.predict(X_test_scaled)
acc = accuracy_score(y_test, y_pred)
print(f"\nDecision Tree Accuracy: {acc:.4f}\n")
print("Classification Report:\n", classification_report(y_test, y_pred))
cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:\n", cm)

# --- Plot confusion matrix (matplotlib) ---
plt.figure(figsize=(5,5))
plt.imshow(cm, interpolation='nearest')
plt.title("Confusion Matrix - Decision Tree")
plt.colorbar()
classes = ['No Rain','Rain'] if set(y_enc) <= {0,1} else [str(c) for c in sorted(set(y_enc))]
plt.xticks(range(len(classes)), classes)
plt.yticks(range(len(classes)), classes)
plt.xlabel('Predicted')
plt.ylabel('Actual')
for i in range(cm.shape[0]):
    for j in range(cm.shape[1]):
        plt.text(j, i, cm[i, j], ha="center", va="center")
plt.show()

# --- Feature importances ---
importances = dt.feature_importances_
indices = np.argsort(importances)[::-1]
top_n = min(20, len(importances))
plt.figure(figsize=(8, max(4, top_n*0.3)))
plt.bar(range(top_n), importances[indices][:top_n])
plt.xticks(range(top_n), [X.columns[i] for i in indices[:top_n]], rotation=90)
plt.title("Top feature importances (Decision Tree)")
plt.tight_layout()
plt.show()

# --- Save model and scaler ---
joblib.dump({'model':dt, 'scaler':scaler, 'features': X.columns.tolist()}, '/content/decision_tree_model.joblib')
print("Saved Decision Tree model to /content/decision_tree_model.joblib")


In [ ]:
# Random Forest training + evaluation (paste into Colab)
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_auc_score, roc_curve
import matplotlib.pyplot as plt
import joblib

# 1) Load dataset (prefer enriched file)
path_enriched = '/content/weather_with_advanced_clusters.csv'
path_orig = '/content/weather_forecast_data.csv'
if os.path.exists(path_enriched):
    df = pd.read_csv(path_enriched)
    print("Loaded enriched dataset:", path_enriched)
elif os.path.exists(path_orig):
    df = pd.read_csv(path_orig)
    print("Loaded base dataset:", path_orig)
else:
    raise FileNotFoundError("No dataset found at /mnt/data. Upload your CSV there.")

# 2) If a standalone 'y' exists in the notebook, attach it to df if df lacks Rain
try:
    if 'y' in globals() and 'Rain' not in df.columns:
        if len(y) == len(df):
            df['Rain'] = y.values
            print("Attached existing 'y' Series to df as df['Rain'].")
        else:
            print("Found 'y' Series but length != df; not attaching automatically.")
except Exception as e:
    print("Skipping automatic attach of 'y' due to:", e)

# 3) Ensure Rain_Encoded exists (encode if necessary)
if 'Rain' in df.columns and 'Rain_Encoded' not in df.columns:
    df['Rain_Encoded'] = df['Rain'].map({'rain':1, 'no rain':0})
    if df['Rain_Encoded'].isnull().any():
        df['Rain_Encoded'] = df['Rain'].astype('category').cat.codes
    print("Created 'Rain_Encoded'. Unique values:", df['Rain_Encoded'].unique())

if 'Rain_Encoded' not in df.columns:
    raise ValueError("No 'Rain_Encoded' target found. Make sure df has a 'Rain' column or a 'y' Series attached.")

# 4) Prepare features and target
target = 'Rain_Encoded'
X = df.drop(columns=[target, 'Rain'], errors='ignore')
X = X.select_dtypes(include=[np.number])  # use numeric features only
y_enc = df[target].astype(int)

print("Features used:", X.columns.tolist())
print("Target distribution:\n", y_enc.value_counts())

# 5) Train/test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y_enc, test_size=0.2, random_state=42, stratify=y_enc
)

# 6) Scaling (not required for RF, but we will keep scaler for consistency)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 7) Train Random Forest (baseline)
rf = RandomForestClassifier(n_estimators=200, random_state=42, n_jobs=-1)
rf.fit(X_train_scaled, y_train)

# 8) Predict & evaluate
y_pred = rf.predict(X_test_scaled)
acc = accuracy_score(y_test, y_pred)
print(f"\nRandom Forest Accuracy: {acc:.4f}\n")
print("Classification Report:\n", classification_report(y_test, y_pred))

cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:\n", cm)

# 9) Confusion matrix plot (matplotlib)
plt.figure(figsize=(5,5))
plt.imshow(cm, interpolation='nearest')
plt.title("Confusion Matrix - Random Forest")
plt.colorbar()
classes = ['No Rain','Rain'] if set(y_enc) <= {0,1} else [str(c) for c in sorted(set(y_enc))]
plt.xticks(range(len(classes)), classes)
plt.yticks(range(len(classes)), classes)
plt.xlabel('Predicted')
plt.ylabel('Actual')
for i in range(cm.shape[0]):
    for j in range(cm.shape[1]):
        plt.text(j, i, cm[i, j], ha="center", va="center")
plt.show()

# 10) ROC AUC (binary case)
if len(set(y_enc)) == 2:
    y_prob = rf.predict_proba(X_test_scaled)[:, 1]
    try:
        auc = roc_auc_score(y_test, y_prob)
        print(f"ROC AUC: {auc:.4f}")
        fpr, tpr, _ = roc_curve(y_test, y_prob)
        plt.figure(figsize=(6,5))
        plt.plot(fpr, tpr)
        plt.plot([0,1],[0,1], linestyle='--')
        plt.title("ROC Curve - Random Forest")
        plt.xlabel("False Positive Rate")
        plt.ylabel("True Positive Rate")
        plt.show()
    except Exception as e:
        print("ROC AUC calculation failed:", e)

# 11) Feature importances (plot top features)
importances = rf.feature_importances_
indices = np.argsort(importances)[::-1]
top_n = min(20, len(importances))

plt.figure(figsize=(8, max(4, top_n*0.3)))
plt.bar(range(top_n), importances[indices][:top_n])
plt.xticks(range(top_n), [X.columns[i] for i in indices[:top_n]], rotation=90)
plt.title("Top feature importances (Random Forest)")
plt.tight_layout()
plt.show()

# 12) Save model + scaler + feature list
model_path = '/content/random_forest_model.joblib'
joblib.dump({'model': rf, 'scaler': scaler, 'features': X.columns.tolist()}, model_path)
print("Saved Random Forest model to:", model_path)

# 13) Show random sample of test rows with predictions
sample_idx = np.random.choice(X_test.index, size=min(10, len(X_test)), replace=False)
sample = X.loc[sample_idx].copy()
sample['true_Rain'] = y_enc.loc[sample_idx].values
sample['pred_Rain'] = rf.predict(scaler.transform(sample[X.columns]))
print("\nSample of test rows with predictions:")
print(sample.head(10).to_string())


In [ ]:
# Fine-tuned MLP with RandomizedSearchCV - paste & run in Colab
import os
import time
import joblib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, StratifiedKFold, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_auc_score, roc_curve
from sklearn.inspection import permutation_importance

# --------- Load dataset ----------
path_enriched = '/content/weather_with_advanced_clusters.csv'
path_orig = '/conent/weather_forecast_data.csv'

if os.path.exists(path_enriched):
    df = pd.read_csv(path_enriched)
    print("Loaded:", path_enriched)
elif os.path.exists(path_orig):
    df = pd.read_csv(path_orig)
    print("Loaded:", path_orig)
else:
    raise FileNotFoundError("No dataset found at /mnt/data. Upload your CSV there.")

# If you have a Series `y` in the notebook and df lacks 'Rain', attach it
try:
    if 'y' in globals() and 'Rain' not in df.columns:
        if len(y) == len(df):
            df['Rain'] = y.values
            print("Attached 'y' Series as df['Rain'].")
        else:
            print("Found 'y' Series but length != df; not attached.")
except Exception as e:
    print("Skipping attach of 'y':", e)

# --------- Ensure target exists ----------
if 'Rain' in df.columns and 'Rain_Encoded' not in df.columns:
    df['Rain_Encoded'] = df['Rain'].map({'rain': 1, 'no rain': 0})
    if df['Rain_Encoded'].isnull().any():
        df['Rain_Encoded'] = df['Rain'].astype('category').cat.codes
    print("Created 'Rain_Encoded'.")

if 'Rain_Encoded' not in df.columns:
    raise ValueError("No 'Rain_Encoded' target found. Make sure df has 'Rain' or attach 'y' Series.")

# --------- Prepare features & target ----------
target = 'Rain_Encoded'
X = df.drop(columns=[target, 'Rain'], errors='ignore')
X = X.select_dtypes(include=[np.number])   # numeric features only
y_enc = df[target].astype(int)

print("Features:", X.columns.tolist())
print("Class counts:\n", y_enc.value_counts())

# --------- Train/test split ----------
X_train, X_test, y_train, y_test = train_test_split(
    X, y_enc, test_size=0.20, random_state=42, stratify=y_enc
)

# --------- Scale features ----------
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# --------- MLP + hyperparameter distribution ----------
mlp = MLPClassifier(max_iter=500, early_stopping=True, n_iter_no_change=20, random_state=42)

param_dist = {
    'hidden_layer_sizes': [(50,), (100,), (50,30), (100,50), (150,100,50)],
    'activation': ['relu', 'tanh'],
    'alpha': [1e-4, 1e-3, 1e-2, 1e-1],
    'learning_rate_init': [1e-3, 5e-4, 1e-4],
    'solver': ['adam', 'sgd'],
    'learning_rate': ['constant', 'adaptive']
}

cv = StratifiedKFold(n_splits=4, shuffle=True, random_state=42)

# n_iter_search: how many parameter settings to try. Reduce to 8-12 for faster runs.
n_iter_search = 24

random_search = RandomizedSearchCV(
    estimator=mlp,
    param_distributions=param_dist,
    n_iter=n_iter_search,
    scoring='f1',
    n_jobs=-1,
    cv=cv,
    random_state=42,
    verbose=2,
    refit=True
)

# --------- Run search ----------
start = time.time()
random_search.fit(X_train_scaled, y_train)
elapsed = time.time() - start
print(f"Search finished in {elapsed:.1f}s. Best CV F1: {random_search.best_score_:.4f}")
print("Best params:", random_search.best_params_)

best_mlp = random_search.best_estimator_

# --------- Evaluate on held-out test set ----------
y_pred = best_mlp.predict(X_test_scaled)
acc = accuracy_score(y_test, y_pred)
print(f"\nTuned MLP Test Accuracy: {acc:.4f}\n")
print("Classification report:\n", classification_report(y_test, y_pred))

cm = confusion_matrix(y_test, y_pred)
print("Confusion matrix:\n", cm)

# --------- Confusion matrix plot (matplotlib) ----------
plt.figure(figsize=(5,5))
plt.imshow(cm, interpolation='nearest')
plt.title("Confusion Matrix - Tuned MLP")
plt.colorbar()
classes = ['No Rain','Rain'] if set(y_enc) <= {0,1} else [str(c) for c in sorted(set(y_enc))]
plt.xticks(range(len(classes)), classes)
plt.yticks(range(len(classes)), classes)
plt.xlabel('Predicted')
plt.ylabel('Actual')
for i in range(cm.shape[0]):
    for j in range(cm.shape[1]):
        plt.text(j, i, cm[i,j], ha='center', va='center')
plt.show()

# --------- ROC & AUC (binary) ----------
if len(set(y_enc)) == 2:
    y_prob = best_mlp.predict_proba(X_test_scaled)[:,1]
    try:
        auc = roc_auc_score(y_test, y_prob)
        print(f"ROC AUC: {auc:.4f}")
        fpr, tpr, _ = roc_curve(y_test, y_prob)
        plt.figure(figsize=(6,5))
        plt.plot(fpr, tpr)
        plt.plot([0,1],[0,1], linestyle='--')
        plt.title('ROC Curve - Tuned MLP')
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.show()
    except Exception as e:
        print("ROC AUC failed:", e)

# --------- Training loss curve (if available) ----------
if hasattr(best_mlp, 'loss_curve_') and len(getattr(best_mlp, 'loss_curve_'))>0:
    plt.figure(figsize=(8,4))
    plt.plot(best_mlp.loss_curve_)
    plt.title('MLP training loss curve')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.show()
else:
    print("No loss_curve_ available for this estimator.")

# --------- Permutation feature importance (on test set) ----------
print("Computing permutation importance (may take a little while)...")
perm = permutation_importance(best_mlp, X_test_scaled, y_test, n_repeats=20, random_state=42, n_jobs=-1)
importances = perm.importances_mean
indices = np.argsort(importances)[::-1]
top_n = min(20, len(importances))

plt.figure(figsize=(8, max(4, top_n*0.3)))
plt.bar(range(top_n), importances[indices][:top_n])
plt.xticks(range(top_n), [X.columns[i] for i in indices[:top_n]], rotation=90)
plt.title('Permutation feature importance (MLP on test set)')
plt.tight_layout()
plt.show()

# --------- Save tuned model ----------
out = {
    'model': best_mlp,
    'scaler': scaler,
    'features': X.columns.tolist(),
    'best_params': random_search.best_params_,
    'cv_results': random_search.cv_results_
}
joblib.dump(out, '/content/tuned_mlp_model.joblib')
print("Saved tuned MLP to /content/tuned_mlp_model.joblib")

# --------- Quick comparison table (if previous models exist) ----------
summary = []
def load_saved(path):
    if os.path.exists(path):
        return joblib.load(path)
    return None

for name, path in [
    ('Decision Tree','/content/decision_tree_model.joblib'),
    ('Random Forest','/content/random_forest_model.joblib'),
    ('LogReg','/content/logistic_regression_model.joblib')]:
    data = load_saved(path)
    if data is not None:
        m = data.get('model')
        s = data.get('scaler')
        f = data.get('features')
        X_test_local = X_test.copy()
        if f is not None:
            X_test_local = X_test[f]
        Xt = s.transform(X_test_local) if s is not None else X_test_local.values
        try:
            p = m.predict(Xt)
            summary.append((name, accuracy_score(y_test, p)))
        except Exception:
            pass

summary.append(('Tuned MLP', acc))
print("\nModel accuracies on the same held-out test set:")
for n,a in summary:
    print(f"{n}: {a:.4f}")

# optional: show a few test rows with MLP predictions
sample_idx = np.random.choice(X_test.index, size=min(8, len(X_test)), replace=False)
sample = X.loc[sample_idx].copy()
sample['true_Rain'] = y_enc.loc[sample_idx].values
sample['pred_Rain_MLP'] = best_mlp.predict(scaler.transform(sample[X.columns]))
print("\nSample MLP predictions:")
print(sample.head(8).to_string())


In [ ]:
# Model comparison cell — paste into Colab and run
import os, joblib, numpy as np, pandas as pd, matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, roc_curve, confusion_matrix

# Paths to saved models (adjust if you saved with different names)
model_paths = {
    'Logistic Regression': '/content/logistic_regression_model.joblib',
    'Decision Tree': '/content/decision_tree_model.joblib',
    'Random Forest': '/content/random_forest_model.joblib',
    'Tuned MLP': '/content/tuned_mlp_model.joblib'
}

# 1) Load dataset
path_enriched = '/content/weather_with_advanced_clusters.csv'
path_orig = '/content/weather_forecast_data.csv'
if os.path.exists(path_enriched):
    df = pd.read_csv(path_enriched)
    print("Loaded:", path_enriched)
elif os.path.exists(path_orig):
    df = pd.read_csv(path_orig)
    print("Loaded:", path_orig)
else:
    raise FileNotFoundError("No dataset found at /mnt/data. Upload your CSV there.")

# If 'y' Series exists in the notebook and df lacks 'Rain', attach it
try:
    if 'y' in globals() and 'Rain' not in df.columns:
        if len(y) == len(df):
            df['Rain'] = y.values
            print("Attached existing 'y' Series as df['Rain'].")
except Exception:
    pass

# Ensure target exists
if 'Rain' in df.columns and 'Rain_Encoded' not in df.columns:
    df['Rain_Encoded'] = df['Rain'].map({'rain':1, 'no rain':0})
    if df['Rain_Encoded'].isnull().any():
        df['Rain_Encoded'] = df['Rain'].astype('category').cat.codes
if 'Rain_Encoded' not in df.columns:
    raise ValueError("No 'Rain_Encoded' target found. Make sure df has 'Rain' or attach 'y' Series.")

# Select numeric features only (drop target and raw 'Rain')
target = 'Rain_Encoded'
X = df.drop(columns=[target, 'Rain'], errors='ignore').select_dtypes(include=[np.number])
y = df[target].astype(int)

# Train/test split (same seed as earlier)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42, stratify=y)

# Global scaler for fallback
global_scaler = StandardScaler().fit(X_train)

def prepare_inputs(saved):
    """
    Given a joblib-loaded object (dict or model), return (X_test_transformed, feature_list, model_obj)
    """
    model_obj = None
    scaler = None
    features = None
    if isinstance(saved, dict):
        model_obj = saved.get('model', None)
        scaler = saved.get('scaler', None)
        features = saved.get('features', None)
    else:
        model_obj = saved

    Xt = X_test.copy()
    if features is not None:
        features_present = [f for f in features if f in Xt.columns]
        Xt = Xt[features_present]
    # scaling
    if scaler is not None:
        try:
            Xt_trans = scaler.transform(Xt)
        except Exception:
            Xt_trans = global_scaler.transform(Xt)
    else:
        try:
            Xt_trans = global_scaler.transform(Xt)
        except Exception:
            Xt_trans = Xt.values
    return Xt_trans, Xt.columns.tolist(), model_obj

# Evaluate each saved model
results = []
roc_curves = {}
for name, path in model_paths.items():
    if not os.path.exists(path):
        print(f"Model file not found (skipping): {name} -> {path}")
        continue
    try:
        saved = joblib.load(path)
    except Exception as e:
        print(f"Failed to load {path}: {e}")
        continue

    X_test_in, features_used, model_obj = prepare_inputs(saved)
    if model_obj is None:
        print(f"No model object found inside {path} (skipping).")
        continue

    # Predictions
    try:
        y_pred = model_obj.predict(X_test_in)
    except Exception as e:
        print(f"Prediction failed for {name}: {e}")
        continue

    # Probabilities for ROC if possible
    y_prob = None
    try:
        if hasattr(model_obj, "predict_proba"):
            y_prob = model_obj.predict_proba(X_test_in)[:,1]
        elif hasattr(model_obj, "decision_function"):
            df_scores = model_obj.decision_function(X_test_in)
            y_prob = (df_scores - df_scores.min()) / (df_scores.max() - df_scores.min() + 1e-12)
    except Exception:
        y_prob = None

    acc = accuracy_score(y_test, y_pred)
    prec = precision_score(y_test, y_pred, zero_division=0)
    rec = recall_score(y_test, y_pred, zero_division=0)
    f1 = f1_score(y_test, y_pred, zero_division=0)
    auc = None
    if y_prob is not None and len(np.unique(y_test)) == 2:
        try:
            auc = roc_auc_score(y_test, y_prob)
            fpr, tpr, _ = roc_curve(y_test, y_prob)
            roc_curves[name] = (fpr, tpr)
        except Exception:
            auc = None

    cm = confusion_matrix(y_test, y_pred)

    results.append({
        'model': name,
        'accuracy': acc,
        'precision_rain': prec,
        'recall_rain': rec,
        'f1_rain': f1,
        'roc_auc': auc if auc is not None else np.nan,
        'confusion_matrix': cm,
        'features_used': features_used
    })

# Build results DataFrame and save
res_df = pd.DataFrame([{
    'Model': r['model'],
    'Accuracy': r['accuracy'],
    'Precision_Rain': r['precision_rain'],
    'Recall_Rain': r['recall_rain'],
    'F1_Rain': r['f1_rain'],
    'ROC_AUC': r['roc_auc']
} for r in results])

res_df.to_csv('/content/model_comparison.csv', index=False)
print("Saved comparison to /content/model_comparison.csv")

# Print or display the results
try:
    from ace_tools import display_dataframe_to_user
    display_dataframe_to_user("Model comparison", res_df)
except Exception:
    print(res_df.to_string())

# ---------- Plots (one figure each) ----------
if not res_df.empty:
    # Accuracy bar chart
    plt.figure(figsize=(6,4))
    plt.bar(res_df['Model'], res_df['Accuracy'])
    plt.title("Model Accuracy")
    plt.ylabel("Accuracy")
    plt.ylim(0,1.02)
    for i,v in enumerate(res_df['Accuracy']):
        plt.text(i, v+0.01, f"{v:.3f}", ha='center')
    plt.show()

    # Precision / Recall / F1 grouped bars
    x = np.arange(len(res_df['Model']))
    width = 0.2
    plt.figure(figsize=(8,4))
    plt.bar(x - width, res_df['Precision_Rain'], width)
    plt.bar(x, res_df['Recall_Rain'], width)
    plt.bar(x + width, res_df['F1_Rain'], width)
    plt.title("Precision / Recall / F1 for 'Rain'")
    plt.xticks(x, res_df['Model'])
    plt.ylim(0,1.02)
    plt.legend(['Precision','Recall','F1'])
    plt.show()

    # ROC curves
    if roc_curves:
        plt.figure(figsize=(6,5))
        for name,(fpr,tpr) in roc_curves.items():
            plt.plot(fpr, tpr, label=name)
        plt.plot([0,1],[0,1], linestyle='--')
        plt.title("ROC Curves")
        plt.xlabel("False Positive Rate")
        plt.ylabel("True Positive Rate")
        plt.legend()
        plt.show()

    # Confusion matrices separately
    for r in results:
        cm = r['confusion_matrix']
        plt.figure(figsize=(4,4))
        plt.imshow(cm, interpolation='nearest')
        plt.title(f"Confusion Matrix - {r['model']}")
        plt.colorbar()
        classes = ['No Rain','Rain'] if set(y) <= {0,1} else [str(c) for c in sorted(set(y))]
        plt.xticks(range(len(classes)), classes)
        plt.yticks(range(len(classes)), classes)
        plt.xlabel('Predicted')
        plt.ylabel('Actual')
        for i in range(cm.shape[0]):
            for j in range(cm.shape[1]):
                plt.text(j, i, cm[i, j], ha='center', va='center')
        plt.show()

print("Comparison complete. model_comparison.csv saved.")
